In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import time
import math

/Users/mustafamuratarat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X_train=pd.read_csv('X_train.txt', header=None).as_matrix()
y_train = pd.read_csv('y_train.txt', header=None).as_matrix().ravel()
X_test = pd.read_csv('X_test.txt', header=None).as_matrix()
y_test = pd.read_csv('y_test.txt', header=None).as_matrix().ravel()

In [3]:
def convertDummy(y, labelNum):
    labelNum = tf.constant(labelNum)
    dummy = tf.one_hot(y, labelNum, axis=1)
    sess = tf.Session()
    dummy = sess.run(dummy)
    sess.close()
    return dummy

In [4]:
def length(sequence):
  used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  sess = tf.Session()
  length = sess.run(length)
  sess.close()
  return length

In [5]:
def miniBatch(x, y, batchSize):
    numObs  = x.shape[0]
    batches = []
    batchNum = math.floor(numObs / batchSize)

    for i in range(batchNum - 1):
        xBatch = x[i * batchSize:(i + 1) * batchSize, :]
        yBatch = y[i * batchSize:(i + 1) * batchSize, :]
        batches.append((xBatch, yBatch))
    xBatch = x[batchNum * batchSize:, :]
    yBatch = y[batchNum * batchSize:, :]
    batches.append((xBatch, yBatch))
    return batches

In [6]:
max_timestep = 60
num_classes = 6
split_size = max_timestep

In [7]:
 def split_time(m, split_size):
        r = m.shape[0]
        extend_row_size = np.math.ceil(r / split_size) * split_size - r
        m_p = np.expand_dims(np.pad(m, [(0, extend_row_size), (0, 0)], mode='constant'), axis=0)
        result = m_p.reshape((np.math.ceil(r / split_size), split_size, m.shape[1]))
        return result

In [8]:
Xtrain = split_time(X_train, split_size)
ytrain = split_time(convertDummy(y_train-1, num_classes), split_size)
Xtest = split_time(X_test, split_size)
ytest = split_time(convertDummy(y_test-1, num_classes), split_size)

In [9]:
print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

(123, 60, 561)
(123, 60, 6)
(50, 60, 561)
(50, 60, 6)


In [10]:
length(Xtrain) #the last instance has only 32 time steps. We left-pad them with zeroes.

array([60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 32], dtype=int32)

In [11]:
learning_rate = 0.01
num_neurons = 128
num_inputs = 561
num_classes = 6
num_steps=60
num_iterations = 100
num_layers=3

#During training, you can feed any value you want to the keep_prob placeholder (typically 0.5)
train_keep_prob =0.5

seq_length_batch = length(Xtrain)

X = tf.placeholder(tf.float32, [None, num_steps, num_inputs])
y = tf.placeholder(tf.int64, [None, None, num_classes])
seq_length = tf.placeholder(tf.int32, [None])

keep_prob = tf.placeholder_with_default(1.0, shape=())

initializer = tf.random_uniform_initializer(-1, 1)
lstm_cells = [tf.contrib.rnn.LSTMCell(num_units=num_neurons, state_is_tuple=True, forget_bias=1.0, initializer=initializer) for layer in range(num_layers)]
lstm_cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob) for cell in lstm_cells]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells_drop)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: Xtrain, seq_length: seq_length_batch, keep_prob: train_keep_prob})

#states variable is a tuple containing one tensor per layer, each representing the final state of that layer's cell
#with shape [batch_size X num_neurons]

#states_val[2] is the tuple for the last layer. It has two arrays in it , one for hidden state h_state, 
#the other for memory state c_state.

#states_val[2][1].shape hidden state has the shape (123, 128), which is [batch_size X num_neurons] 
#that we mentioned above.


#the hidden state of the last layer in states tuple will be the same with the last time-step of outputs 
#for one particular observation

In [35]:
states_val[2]

LSTMStateTuple(c=array([[-6.7680970e-02,  9.1396743e-01,  9.9923646e-01, ...,
         1.0229808e+00,  1.7731071e+00,  2.4362956e-01],
       [ 1.4840257e+00, -1.6282718e+00,  1.4953713e+00, ...,
        -8.5468493e-02, -1.2305751e+00, -8.5696262e-01],
       [-9.4343626e-01,  5.9862161e-01, -5.7227069e-01, ...,
         2.3866761e-02, -4.5293739e-01,  1.3368111e-03],
       ...,
       [-9.2974854e-01, -2.0055228e-01,  7.7674824e-01, ...,
         4.9443191e-01,  8.8370866e-01, -2.2136122e-03],
       [ 3.9246514e-02, -1.2646917e-01, -2.0961554e-01, ...,
        -2.9355648e-01, -1.0848233e-01, -4.5372680e-02],
       [ 4.8759878e-02, -4.2822036e-01,  4.2377287e-01, ...,
        -9.1928977e-01,  1.5365555e+00,  8.9523041e-01]], dtype=float32), h=array([[-1.4614449e-03,  4.1632261e-03,  6.1467177e-01, ...,
         7.7088445e-01,  4.4296119e-01,  2.3714852e-01],
       [ 6.7463249e-01, -1.3774876e-01,  7.5513667e-01, ...,
        -4.3386869e-02, -7.5190824e-01, -1.2458118e-01],
       [

In [23]:
outputs_val[0][59]

array([-1.46144489e-03,  4.16322611e-03,  6.14671767e-01,  5.14349900e-02,
        9.80566163e-03,  1.13957962e-04, -7.48810351e-01, -3.20786843e-04,
        7.59860426e-02,  5.62000394e-01, -9.23652470e-01, -1.01787165e-01,
       -1.71003165e-03, -4.18778621e-02, -5.71841121e-01,  4.38588113e-03,
        4.09034640e-01, -7.33205210e-03, -5.77882957e-03, -5.62575579e-01,
       -8.00679445e-01,  6.86107039e-01, -6.51394352e-02, -2.44840831e-01,
        4.28312629e-01, -2.79166661e-02,  2.77712375e-01,  6.22436055e-05,
       -2.19905391e-01, -1.78232878e-01, -2.94782333e-02, -2.70124108e-01,
        1.37052964e-03, -2.62694862e-02, -7.13243634e-02, -6.28041178e-02,
        8.88245165e-01,  8.26203376e-02,  5.75758219e-01,  6.21252418e-01,
        4.51927044e-05, -8.82703602e-01,  8.04276109e-01,  1.94275672e-05,
       -8.63878667e-01, -2.24489853e-01, -1.59137268e-02,  4.58719939e-01,
        1.20201118e-01, -7.36764193e-01, -2.04624087e-02,  6.41722322e-01,
        4.19830641e-04, -

In [33]:
states_val[2][1][0]

array([-1.46144489e-03,  4.16322611e-03,  6.14671767e-01,  5.14349900e-02,
        9.80566163e-03,  1.13957962e-04, -7.48810351e-01, -3.20786843e-04,
        7.59860426e-02,  5.62000394e-01, -9.23652470e-01, -1.01787165e-01,
       -1.71003165e-03, -4.18778621e-02, -5.71841121e-01,  4.38588113e-03,
        4.09034640e-01, -7.33205210e-03, -5.77882957e-03, -5.62575579e-01,
       -8.00679445e-01,  6.86107039e-01, -6.51394352e-02, -2.44840831e-01,
        4.28312629e-01, -2.79166661e-02,  2.77712375e-01,  6.22436055e-05,
       -2.19905391e-01, -1.78232878e-01, -2.94782333e-02, -2.70124108e-01,
        1.37052964e-03, -2.62694862e-02, -7.13243634e-02, -6.28041178e-02,
        8.88245165e-01,  8.26203376e-02,  5.75758219e-01,  6.21252418e-01,
        4.51927044e-05, -8.82703602e-01,  8.04276109e-01,  1.94275672e-05,
       -8.63878667e-01, -2.24489853e-01, -1.59137268e-02,  4.58719939e-01,
        1.20201118e-01, -7.36764193e-01, -2.04624087e-02,  6.41722322e-01,
        4.19830641e-04, -